# Word2Vec: Interpretação da linguagem humana com word embedding
## Classificação de notícias

### Bibliotecas básicas e outras inicializações

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
%matplotlib inline

### Carregamento dos dados

In [2]:
artigo_treino = pd.read_csv('./dataset/treino.csv')
print(artigo_treino.shape)
artigo_treino.sample(5)

(90000, 6)


,title,text,date,category,subcategory,link
21725,\n\t\tMinistro reafirma continuidade do Bolsa ...,"DO UOL O ministro do Esporte, Leonardo Piccia...",2016-05-10,esporte,olimpiada-no-rio,http://www1.folha.uol.com.br/esporte/olimpiada...
53852,Sérgio Reis defende redução da maioridade pena...,"Ainda mais radical que alguns de seus colegas,...",2015-04-28,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/04...
10913,Justiça nega recurso e mantém suspensa cobranç...,A Justiça Federal negou nesta terça-feira (14)...,2017-03-14,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2017/03...
930,Nova primeira-dama de SP diz que Doria fará 'm...,"Ladeada de amigas e seguranças, a nova primeir...",2016-10-24,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2016/10...
28643,Coleção apresenta os sabores picantes da culin...,A cozinha colorida e condimentada do México é ...,2016-07-17,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2016/07...


In [3]:
artigo_teste = pd.read_csv('./dataset/teste.csv')
print(artigo_teste.shape)
artigo_teste.sample(5)

(20513, 6)


,title,text,date,category,subcategory,link
12947,EUA estabelecem margem de dumping para importa...,O Departamento de Comércio dos Estados Unidos ...,2016-12-01,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/01/1...
16278,Palmeiras busca 1ª vitória pelo Brasileiro na ...,"O Palmeiras busca contra o Goiás, neste doming...",2015-05-24,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/05/1...
18731,"Em clima de campanha, prefeito Eduardo Paes re...","Em clima de campanha antecipada, o prefeito do...",2015-06-09,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/09...
18405,Trump é fruto inesperado dos desvios dos liber...,"""Um liberal não passa de um fascista em férias...",2016-12-31,colunas,demetriomagnoli,http://www1.folha.uol.com.br/colunas/demetriom...
2430,"Fato inédito nos pontos corridos, título sem j...",Os jogadores do Corinthians podem ser os prime...,2015-04-11,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/11/1...


### Modelo pré-treinado

Fonte: http://www.nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc

In [4]:
from gensim.models import KeyedVectors
modelo = KeyedVectors.load_word2vec_format('./embbedings/skip_s300.txt')

modelo.most_similar('china')

[('índia', 0.7630469799041748),
 ('taiwan', 0.7408342361450195),
 ('tailândia', 0.710228443145752),
 ('indonésia', 0.6836247444152832),
 ('malásia', 0.6769840717315674),
 ('mongólia', 0.6427473425865173),
 ('coreia', 0.6382735967636108),
 ('manchúria', 0.6369147896766663),
 ('singapura', 0.6189110279083252),
 ('rússia', 0.5982829332351685)]

In [5]:
modelo.most_similar(positive=['brasil', 'argentina'])

[('chile', 0.7519630193710327),
 ('paraguai', 0.6889568567276001),
 ('peru', 0.6882554292678833),
 ('uruguai', 0.6853610873222351),
 ('venezuela', 0.6830991506576538),
 ('bolívia', 0.6790730953216553),
 ('méxico', 0.6655052900314331),
 ('colômbia', 0.664671003818512),
 ('equador', 0.6520291566848755),
 ('panamá', 0.5880077481269836)]

In [6]:
modelo.most_similar(positive=['médico', 'mulher'], negative=['homem'])

[('enfermeira', 0.6510195732116699),
 ('obstetra', 0.6261360049247742),
 ('pediatra', 0.5951809883117676),
 ('neonatologista', 0.5767174959182739),
 ('anestesista', 0.5706735849380493),
 ('ginecologista', 0.5536660552024841),
 ('fisioterapeuta', 0.5470119714736938),
 ('voluti', 0.5362079739570618),
 ('nutricionista', 0.5361762046813965),
 ('fonoaudióloga', 0.5353548526763916)]

### Vetorização do texto

In [7]:
import nltk
import string

def tokenizador(texto):
    texto = texto.lower()
    lista_alfanumerico = []
    
    for token_valido in nltk.word_tokenize(texto):
        if token_valido in string.punctuation: continue
        lista_alfanumerico.append(token_valido)        
    return lista_alfanumerico

tokenizador(artigo_treino['title'][0])

['após',
 'polêmica',
 'marine',
 'le',
 'pen',
 'diz',
 'que',
 'abomina',
 'negacionistas',
 'do',
 'holocausto']

### Combinando vetores

In [12]:
def combinacao_vetores_por_soma(palavras_numeros):
    vetor_resultante = np.zeros(300)
    for pn in palavras_numeros:
        try:
            vetor_resultante += modelo.get_vector(pn)
        except KeyError:
            if pn.isnumeric():
                pn = '0'*len(pn)
                vetor_resultante += modelo.get_vector(pn)
            else:
                vetor_resultante += modelo.get_vector('unknown')                
    return vetor_resultante

vetor_resultante = combinacao_vetores_por_soma(tokenizador(artigo_treino['title'][0]))
vetor_resultante[:10]

array([-0.86259399, -0.09650198, -0.04621498,  0.062894  , -0.42186402,
        0.949122  , -1.08391203,  0.30013298, -0.34467998, -0.77671101])

### Dados de treino e teste

In [17]:
def matriz_vetores(textos):
    x = len(textos)
    y = 300
    matriz = np.zeros((x, y))
    
    for i in range(x):
        palavras_numeros = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_vetores_por_soma(palavras_numeros)
    return matriz

matriz_vetores_treino = matriz_vetores(artigo_treino['title'])
matriz_vetores_teste = matriz_vetores(artigo_teste['title'])

In [18]:
print(matriz_vetores_treino.shape)
print(matriz_vetores_teste.shape)

(90000, 300)
(20513, 300)


### Classificação

In [28]:
from sklearn.linear_model import LogisticRegression
modelo = LogisticRegression(solver='lbfgs', max_iter=1000, penalty='l2', tol=0.0001, C=1.0)
modelo.fit(matriz_vetores_treino, artigo_treino['category'])

LogisticRegression(max_iter=1000)

In [29]:
modelo.n_iter_

array([245], dtype=int32)

In [30]:
modelo.score(matriz_vetores_teste, artigo_teste['category'])

0.806805440452396

#### Predição

In [31]:
y_pred = modelo.predict(matriz_vetores_teste)

#### Interpretando o resultado

In [32]:
from sklearn.metrics import classification_report
print(classification_report(artigo_teste['category'], y_pred))

              precision    recall  f1-score   support

     colunas       0.86      0.72      0.78      6103
   cotidiano       0.63      0.81      0.70      1698
     esporte       0.93      0.89      0.91      4663
   ilustrada       0.15      0.91      0.26       131
     mercado       0.84      0.81      0.83      5867
       mundo       0.76      0.86      0.80      2051

    accuracy                           0.81     20513
   macro avg       0.69      0.83      0.71     20513
weighted avg       0.84      0.81      0.82     20513



### Comparando classificadores

In [37]:
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier()
dummy.fit(matriz_vetores_treino, artigo_treino['category'])

DummyClassifier()

In [39]:
y_pred = dummy.predict(matriz_vetores_teste)
print(classification_report(artigo_teste['category'], y_pred, zero_division=0))

              precision    recall  f1-score   support

     colunas       0.30      1.00      0.46      6103
   cotidiano       0.00      0.00      0.00      1698
     esporte       0.00      0.00      0.00      4663
   ilustrada       0.00      0.00      0.00       131
     mercado       0.00      0.00      0.00      5867
       mundo       0.00      0.00      0.00      2051

    accuracy                           0.30     20513
   macro avg       0.05      0.17      0.08     20513
weighted avg       0.09      0.30      0.14     20513

